# LTSM

based on https://machinelearningmastery.com/text-generation-lstm-recurrent-neural-networks-python-keras/

In [2]:
import sys
#!conda install --yes --prefix {sys.prefix} keras

In [3]:
# load packages
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils

In [4]:
# load text and covert to lowercase
filename = "Bowie_10K.txt"
raw_text = open(filename, 'r', encoding='utf-8').read()
raw_text = raw_text.lower()

In [5]:
# create mapping of unique chars to integers, and a reverse mapping
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [6]:
# summarize the loaded data
n_chars = len(raw_text)
n_vocab = len(chars)
print("Total Characters: ", n_chars)
print("Total Vocab: ", n_vocab)
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
    seq_in = raw_text[i:i + seq_length]
    seq_out = raw_text[i + seq_length]
    dataX.append([char_to_int[char] for char in seq_in])
    dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print("Total Patterns: ", n_patterns)




Total Characters:  293525
Total Vocab:  58
Total Patterns:  293425


In [7]:
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)

In [8]:
# define the LSTM model with Keras
# If you can, try CuDNNLSTM
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')
# define the checkpoint
filepath="weights-improvement3-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [9]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 256)               264192    
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense (Dense)                (None, 58)                14906     
Total params: 279,098
Trainable params: 279,098
Non-trainable params: 0
_________________________________________________________________


In [9]:
# fit the model
model.fit(X, y, epochs=15, batch_size=128, callbacks=callbacks_list)

Epoch 1/15
2293/2293 [==============================] - ETA: 0s - loss: 2.9000
Epoch 00001: loss improved from inf to 2.90000, saving model to weights-improvement3-01-2.9000.hdf5
2293/2293 [==============================] - 1110s 484ms/step - loss: 2.9000
Epoch 2/15
2293/2293 [==============================] - ETA: 0s - loss: 2.7400
Epoch 00002: loss improved from 2.90000 to 2.74003, saving model to weights-improvement3-02-2.7400.hdf5
2293/2293 [==============================] - 1100s 480ms/step - loss: 2.7400
Epoch 3/15
2293/2293 [==============================] - ETA: 0s - loss: 2.6505
Epoch 00003: loss improved from 2.74003 to 2.65045, saving model to weights-improvement3-03-2.6505.hdf5
2293/2293 [==============================] - 1100s 480ms/step - loss: 2.6505
Epoch 4/15
2293/2293 [==============================] - ETA: 0s - loss: 2.5551
Epoch 00004: loss improved from 2.65045 to 2.55513, saving model to weights-improvement3-04-2.5551.hdf5
2293/2293 [==============================

In [38]:
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
" use it's you that sets the test.
so much has gone and little is new
and as the sunrise stream
flicke "
 the soane so the soane she soane sh the soane so meve i want to beeye io the woudg dud yhu wou dan to benee to tee wour  aods cor the soane so toe toane io the soane so the toine io the woudg dud yhu wou dan te ben tee soane io the woudd du s tou late to be late
it a mase
oo th  love it al tht to meve io a bas
aall ih a mast
far i lane to be late
it a mast
oo the fire
oo the five
oo m man  io soe fove
oo mo toe five
in a mast
and the soane to ben tee soane to ben tee soane io the woudd dud yhu wou dan to benee to tee wour  aods cor the soane so toe toane io the soane so the toine io the woudg dud yhu wou dan te ben tee soane io the woudd du s tou late to be late
it a mase
oo th  love it al tht to meve io a bas
aall ih a mast
far i lane to be late
it a mast
oo the fire
oo the five
oo m man  io soe fove
oo mo toe five
in a mast
and the soane to ben tee soane to ben tee soane io

In [34]:
# define the LSTM model with Keras
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))

# load the network weights
filename = "./weights-improvement3-05-2.4481.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')

# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print("Seed:")
print("\"", ''.join([int_to_char[value] for value in pattern]), "\"")

# generate characters
for i in range(1000):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print("\nDone.")

Seed:
" don: 15 million 75 thousand.
new york: 80 million.
paris: 15 million and 30.
china: 1000 million.
bi "
d ihse
ih mas sh mh the soane so the toane to meve i want to banee tot to the soane so the toine io the woudg dud yhu wou dan te ben tee soane io the woudd du s tou late to be late
it a mase
oo th  love it al tht to meve io a bas
aall ih a mast
far i lane to be late
it a mast
oo the fire
oo the five
oo m man  io soe fove
oo mo toe five
in a mast
and the soane to ben tee soane to ben tee soane io the woudd dud yhu wou dan to benee to tee wour  aods cor the soane so toe toane io the soane so the toine io the woudg dud yhu wou dan te ben tee soane io the woudd du s tou late to be late
it a mase
oo th  love it al tht to meve io a bas
aall ih a mast
far i lane to be late
it a mast
oo the fire
oo the five
oo m man  io soe fove
oo mo toe five
in a mast
and the soane to ben tee soane to ben tee soane io the woudd dud yhu wou dan to benee to tee wour  aods cor the soane so toe toane io